In [ ]:
!pip install python-binance

In [ ]:
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=c450cad7107846b593a8719c5273902db69887b8692e358e944a52a0a2ee8891
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a1fa7fb012667e55cb0643cfa65996bddecc
Successfully built ta


In [ ]:
from binance import Client
import pandas as pd
import ta
import numpy as np
import time
import datetime

api_key = "ncaGKCQNu9KS8kEwF3Jri37YjsOkKCfLGiKZcq87iThEU3QZIWdxOyeQLUvOCPpG"
api_secret = "tvSQ36Idx7W1QwiMx5ycduhziLQZaHUbhnHZGhqlSRaF6VrkP8QqsLY3LCidthdJ"

client = Client(api_key, api_secret)

In [ ]:
def getminutedata(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' min ago KST'))
  frame = frame.iloc[:, :6]
  frame.columns  = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit='ms')
  frame = frame.astype(float)
  time_change = datetime.timedelta(hours=9)
  frame.index = frame.index + time_change
  return frame

In [ ]:
df = getminutedata('BTCUSDT', '1m', '100')

In [ ]:
def applytechnicals(df):
  df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window=14, smooth_window=3)
  df['%D'] = df['%K'].rolling(3).mean()
  df['rsi'] = ta.momentum.rsi(df.Close, window=14)
  df['macd'] = ta.trend.macd_diff(df.Close)
  df.dropna(inplace=True)

In [ ]:
applytechnicals(df)

In [ ]:
df

,Open,High,Low,Close,Volume,%K,%D,rsi,macd
Time,,,,,,,,,
2022-06-05 22:32:00,29684.44,29685.14,29682.74,29685.13,8.31902,11.642822,10.997736,23.677706,-3.409938
2022-06-05 22:33:00,29685.13,29700.49,29685.13,29700.48,13.32660,36.885381,19.684262,39.446019,-2.359594
2022-06-05 22:34:00,29700.48,29700.49,29700.48,29700.48,3.22621,45.691587,31.406596,39.446019,-1.503806
2022-06-05 22:35:00,29700.49,29700.49,29695.29,29695.29,7.78849,40.612485,41.063151,36.489818,-1.142609
2022-06-05 22:36:00,29695.29,29703.17,29695.29,29703.16,6.43525,59.814197,48.706090,43.414899,-0.274284
...,...,...,...,...,...,...,...,...,...
2022-06-05 23:34:00,29768.13,29770.78,29762.78,29762.79,5.99118,31.887241,40.240556,60.252456,-1.980372
2022-06-05 23:35:00,29762.79,29770.77,29761.80,29768.41,6.22128,42.174629,38.568552,62.182837,-2.096594
2022-06-05 23:36:00,29768.40,29768.41,29767.57,29767.57,1.87614,49.742326,41.268066,61.700501,-2.292469


In [ ]:
class Signals:

  def __init__(self, df, lags):
    self.df = df
    self.lags = lags

  def gettrigger(self):
    dfx = pd.DataFrame()
    for i in range(self.lags + 1):
      mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
      dfx = dfx.append(mask, ignore_index=True)
    return dfx.sum(axis=0)

  def decide(self):
      self.df['trigger'] = np.where(self.gettrigger(), 1, 0)
      self.df['Buy'] = np.where((self.df.trigger) & (self.df['%K'].between(20, 80)) &
                                (self.df['%D'].between(20, 80)) & (self.df.rsi > 50) & (self.df.macd > 0), 1, 0)

In [ ]:
inst = Signals(df, 25)

In [ ]:
inst.decide()

In [ ]:
df

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2022-06-05 22:32:00,29684.44,29685.14,29682.74,29685.13,8.31902,11.642822,10.997736,23.677706,-3.409938,1,0
2022-06-05 22:33:00,29685.13,29700.49,29685.13,29700.48,13.32660,36.885381,19.684262,39.446019,-2.359594,1,0
2022-06-05 22:34:00,29700.48,29700.49,29700.48,29700.48,3.22621,45.691587,31.406596,39.446019,-1.503806,1,0
2022-06-05 22:35:00,29700.49,29700.49,29695.29,29695.29,7.78849,40.612485,41.063151,36.489818,-1.142609,1,0
2022-06-05 22:36:00,29695.29,29703.17,29695.29,29703.16,6.43525,59.814197,48.706090,43.414899,-0.274284,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-05 23:34:00,29768.13,29770.78,29762.78,29762.79,5.99118,31.887241,40.240556,60.252456,-1.980372,0,0
2022-06-05 23:35:00,29762.79,29770.77,29761.80,29768.41,6.22128,42.174629,38.568552,62.182837,-2.096594,0,0
2022-06-05 23:36:00,29768.40,29768.41,29767.57,29767.57,1.87614,49.742326,41.268066,61.700501,-2.292469,0,0


In [ ]:
df[df.Buy == 1]

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2022-06-05 22:51:00,29731.45,29731.45,29724.44,29724.44,5.88375,63.353387,76.274813,53.926117,3.039990,1,1
2022-06-05 23:26:00,29770.36,29770.36,29747.11,29750.56,15.69603,62.630385,76.787604,59.404232,5.486469,1,1
2022-06-05 23:27:00,29750.55,29750.56,29749.06,29749.06,2.74057,61.496599,67.241119,58.809359,2.765918,1,1
2022-06-05 23:28:00,29749.06,29759.50,29745.37,29756.85,11.19114,66.448954,63.525313,60.993947,1.216766,1,1


In [ ]:
def strategy(pair, qty, open_position=False):
  df = getminutedata(pair, '1m', '100')
  applytechnicals(df)
  inst = Signals(df, 25)
  inst.decide()
  print(f'current Close is ' + str(df.Close.iloc[-1]))
  if df.Buy.iloc[-1]:
    order = client.create_order(symbol=pair, side='BUY', type='MARKET', quantity=qty)
    print(order)
    buyprice = order['fills'][0]['price']
    open_position = True

  while open_position:
    time.sleep(0.5)
    df = getminutedata(pair, '1m', '2')
    print(f'current Close' + str(df.Close.iloc[-1]))
    print(f'current Target' + str(buyprice * 1.005))
    print(f'current Stop is' + str(buyprice * 0.995))

    if df.Close[-1] <= buyprice * 0.995  or df.Close[-1] >= buyprice * 1.005:
      order = client.create_order(symbol=pair, side='SELL', type='MARKET', quantity=qty)
      print(order)
      break

In [ ]:
while True:
  strategy('BTCUSDT', 50)
  time.sleep(0.5)